In [ ]:
!apt-get update

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,284 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,158 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,264 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.8 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [998 kB]
Hit:13 https:

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
# import findspark
# findspark.init()
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()
# spark

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [112]:
import pandas as pd
from pyspark.sql.functions import mean,when,lit,max
import numpy as np

In [5]:
mydata = spark.read.csv('/content/original.csv',header=True,inferSchema=True)

In [6]:
mydata.printSchema()

root
 |-- id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- City: string (nullable = true)
 |-- JobTitle: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)



In [7]:
mydata.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

#### Example of doing mean of values

In [8]:
mydata.select(mean(mydata['Latitude']).alias('average')).show()

+-----------------+
|          average|
+-----------------+
|25.43151724234234|
+-----------------+



#### There are few null values in "City" column. Replace null values by unknown.

In [35]:
mydata = mydata.na.fill('Unknown',subset=['City'])

In [36]:
mydata.head(2)

[Row(id=1, first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude=50.5774075, Longitude=16.4967184),
 Row(id=2, first_name='Kimberly', last_name='Von Welden', gender='Female', City='Bulgan', JobTitle='Programmer II', Salary='$62846.60', Latitude=48.8231572, Longitude=103.5218199)]

Create a new dataframe that has a new column Null vlues filled with Unknown in City column

In [37]:
mydata2 = mydata.withColumn('clean_city', when(mydata.City.isNull(),'Unknown').otherwise(mydata.City))

Overwrite the newly created dataframe with JobTitle = Not Null.  

In [38]:
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
# Approach- 1
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
mydata2 = mydata2.na.drop(subset=['JobTitle'])

In [39]:
#:::::::::::::::::::::::::::::::::::::::::::::::
# Approach- 2
#:::::::::::::::::::::::::::::::::::::::::::::::
# mydata2 = mydata2.filter(mydata.JobTitle.isNotNull()).show()

#### Replace the Salary is Null with mean value

Salary column ins string in schema. However, we need to remove the leading dollar signature from the column. We will use substring method.

In [40]:
mydata2 = mydata2.withColumn('clean_salary',mydata2.Salary.substr(2,100).cast('float'))
## The last cast command has been added, because the type of Salary was str and only rmove the dollar sign will not convert it to str,
# hence we need to forcefully convert to float.

In [41]:
salary_mean = mydata2.select(mean(mydata2['clean_salary'])).collect()[0][0]

#### Now replace Null values by using the above mean

In [42]:
## @@ Method-1
#mydata2 = mydata2.na.fill(salary_mean,subset=['clean_salary'])

## @@ Method-2
mydata2 = mydata2.withColumn('clean_salary',when(mydata2.clean_salary.isNull(),lit(salary_mean)).otherwise(mydata2.clean_salary))

#### Replace the Null values of Latitude column with median.

In [43]:
## Unlike mean function, there is no such inbuild functionality like median. So, we need to call numpy for median calculation.
# the process is given below.
latitude = mydata2.select(['Latitude'])

In [44]:
lat_median = np.median(latitude.na.drop().collect())

In [45]:
mydata2 = mydata2.na.fill(lat_median,subset = ['Latitude'])

#### Gender-wise salary

In [85]:
### Genderwise average salary
gender_salary = mydata2.groupBy('gender').agg(mean('clean_salary').alias('avg_salary'))

In [69]:
### Create two seperate columns for male and female salary and put 0 for not female and male respectively.
df = mydata2.withColumn('female_salary',when(mydata2.gender == 'Female',mydata2.clean_salary).otherwise(0))

df = df.withColumn('male_salary',when(df.gender == 'Male',df.clean_salary).otherwise(0))

In [86]:
### Salary by Jobtitle and gender.
salary_by_title_gender = df.groupBy(['JobTitle','gender']).mean('clean_salary')

In [81]:
salary_by_title_gender.filter(salary_by_title_gender['JobTitle'] == 'Staff Accountant IV').show()

+-------------------+------+------------------+
|           JobTitle|gender| avg(clean_salary)|
+-------------------+------+------------------+
|Staff Accountant IV|  Male|      73921.328125|
|Staff Accountant IV|Female|52926.352864583336|
+-------------------+------+------------------+



In [84]:
salary_by_title_gender.orderBy('JobTitle').show()

+--------------------+------+------------------+
|            JobTitle|gender| avg(clean_salary)|
+--------------------+------+------------------+
| Account Coordinator|  Male|51446.623697916664|
| Account Coordinator|Female|     46707.4453125|
|   Account Executive|  Male|    65415.96796875|
|   Account Executive|Female|   52020.779296875|
|Account Represent...|Female|51116.973307291664|
|Account Represent...|Female|    41786.91015625|
|Account Represent...|  Male|    40562.69921875|
|Account Represent...|  Male|     22420.7109375|
|       Accountant II|Female|  50354.8154296875|
|      Accountant III|Female|  15589.5595703125|
|      Accountant III|  Male|39183.798177083336|
|       Accountant IV|Female|   82732.248046875|
|Accounting Assist...|  Male|     59255.4296875|
|Accounting Assist...|Female|  58916.0830078125|
|Accounting Assist...|  Male|   18795.439453125|
|Accounting Assist...|Female|44071.866536458336|
|Accounting Assist...|Female|      57337.484375|
|             Actuar

In [89]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+----------------+----------------+----------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|    clean_salary|   female_salary|     male_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+----------------+----------------+----------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|   57438.1796875|   57438.1796875|             0.0|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|   62846.6015625|   62846.6015625|             0.0|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|  61489.23046875|   

In [104]:
### Create a column that will have salary difference for Male and Female for each JobTitle.

salary_by_title_gender = df.groupBy('JobTitle').agg(mean('male_salary').alias('final_male_salary'),
                                                    mean('female_salary').alias('final_female_salary'))

In [107]:
salary_by_title_gender.withColumn('delta',salary_by_title_gender.final_male_salary-salary_by_title_gender.final_female_salary).show()

+--------------------+------------------+-------------------+-------------------+
|            JobTitle| final_male_salary|final_female_salary|              delta|
+--------------------+------------------+-------------------+-------------------+
|Systems Administr...|  15540.9501953125|    50590.474609375|  -35049.5244140625|
|   Media Manager III|17381.920572916668| 29586.436197916668|      -12204.515625|
|  Recruiting Manager|  26383.4951171875| 34848.452473958336| -8464.957356770836|
|       Geologist III|    12830.75390625|       31749.046875|    -18918.29296875|
|        Geologist II|   43293.865234375|                0.0|    43293.865234375|
|Database Administ...|     52018.4609375|                0.0|      52018.4609375|
|   Financial Analyst|       39606.05625|    23353.776953125|    16252.279296875|
|  Analyst Programmer|  21042.9634765625|   16406.1287109375|  4636.834765625001|
|Software Engineer II|      74782.640625|                0.0|       74782.640625|
|       Accounta

### Which city has the highest average salary

In [120]:
dff = mydata2.groupby('clean_city').agg(mean('clean_salary').alias('avg_salary'))

In [132]:
max_sal = dff.select(max(dff.avg_salary)).collect()[0][0]

In [134]:
dff.filter(dff.avg_salary == max_sal).show()

+----------+-----------+
|clean_city| avg_salary|
+----------+-----------+
| Mesopotam|99948.28125|
+----------+-----------+

